## code to download images

In [1]:
import os
import pip._vendor.requests as requests
from typing import List
import json

class PokemonCardData:

    def __init__(self, id, name, image_url):
        self.id = id
        self.name = name
        self.image_url = image_url

def download_image(url, file_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Image successfully downloaded: {file_path}")
    else:
        print(f"Failed to retrieve image. Status code: {response.status_code}")

def read_json_array(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
        return data
    
def extract_values(json_array):
    card_array = []
    for item in json_array:
        id = item.get('id')
        name = item.get('name')
        image_url = item.get('images', {}).get('small')
        rarity = item.get('rarity')

        if rarity is not None:
            if 'rare' in rarity.lower():
                card = PokemonCardData(id, name, image_url)
                card_array.append(card)
    
    return card_array

def download_cards(card_array : List[PokemonCardData], folder_path):
    for card in card_array:
        file_path = os.path.join(folder_path, card.id + '.png')
        download_image(card.image_url, file_path)

In [11]:
def download_all_images(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            data = read_json_array(file_path)
            card_array = extract_values(data)
            download_cards(card_array, 'pokemon-tcg-images')


In [13]:
# download_all_images('pokemon-tcg-data/cards/en')

Image successfully downloaded: pokemon-tcg-images/sv1-15.png
Image successfully downloaded: pokemon-tcg-images/sv1-19.png
Image successfully downloaded: pokemon-tcg-images/sv1-23.png
Image successfully downloaded: pokemon-tcg-images/sv1-32.png
Image successfully downloaded: pokemon-tcg-images/sv1-38.png
Image successfully downloaded: pokemon-tcg-images/sv1-41.png
Image successfully downloaded: pokemon-tcg-images/sv1-43.png
Image successfully downloaded: pokemon-tcg-images/sv1-45.png
Image successfully downloaded: pokemon-tcg-images/sv1-54.png
Image successfully downloaded: pokemon-tcg-images/sv1-61.png
Image successfully downloaded: pokemon-tcg-images/sv1-65.png
Image successfully downloaded: pokemon-tcg-images/sv1-76.png
Image successfully downloaded: pokemon-tcg-images/sv1-80.png
Image successfully downloaded: pokemon-tcg-images/sv1-81.png
Image successfully downloaded: pokemon-tcg-images/sv1-86.png
Image successfully downloaded: pokemon-tcg-images/sv1-88.png
Image successfully downl

In [14]:
def count_files_in_folder(folder_path):
    # Ensure the path is a directory
    if not os.path.isdir(folder_path):
        return 0
    
    # Count the number of files
    num_files = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
    
    return num_files

In [15]:
print(count_files_in_folder('pokemon-tcg-images/'))

7437


In [2]:
import shutil

def organize_images(directory):
    # Ensure the provided directory exists
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist.")
        return

    # Loop through all files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # Check if the file is an image (you can extend the list of extensions as needed)
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp')):
            # Create a new directory for the image
            image_name = os.path.splitext(filename)[0]
            new_directory = os.path.join(directory, image_name)
            
            # Ensure the new directory does not already exist
            if not os.path.exists(new_directory):
                os.makedirs(new_directory)
            
            # Move the image into the new directory
            new_path = os.path.join(new_directory, filename)
            shutil.move(file_path, new_path)
            print(f"Moved {filename} to {new_directory}")

In [4]:
# organize_images('pokemon-tcg-images')

Moved hgss4-93.png to pokemon-tcg-images/hgss4-93
Moved sv3-124.png to pokemon-tcg-images/sv3-124
Moved swsh4-98.png to pokemon-tcg-images/swsh4-98
Moved sv4-242.png to pokemon-tcg-images/sv4-242
Moved dp4-30.png to pokemon-tcg-images/dp4-30
Moved swsh12pt5-100.png to pokemon-tcg-images/swsh12pt5-100
Moved sm5-93.png to pokemon-tcg-images/sm5-93
Moved sm3-164.png to pokemon-tcg-images/sm3-164
Moved ex10-114.png to pokemon-tcg-images/ex10-114
Moved swsh3-77.png to pokemon-tcg-images/swsh3-77
Moved sm1-139.png to pokemon-tcg-images/sm1-139
Moved xy1-1.png to pokemon-tcg-images/xy1-1
Moved sv4pt5-92.png to pokemon-tcg-images/sv4pt5-92
Moved col1-39.png to pokemon-tcg-images/col1-39
Moved sma-SV38.png to pokemon-tcg-images/sma-SV38
Moved sm7-65.png to pokemon-tcg-images/sm7-65
Moved sv4-100.png to pokemon-tcg-images/sv4-100
Moved ex6-107.png to pokemon-tcg-images/ex6-107
Moved bw3-12.png to pokemon-tcg-images/bw3-12
Moved ecard2-1.png to pokemon-tcg-images/ecard2-1
Moved swsh10-13.png to p

In [6]:
import imagehash
from PIL import Image
base1img = Image.open('pokemon-tcg-images/base1-4.png')
phash = imagehash.phash(base1img)
dhash = imagehash.dhash(base1img)
whash = imagehash.whash(base1img)
chash = imagehash.crop_resistant_hash(base1img)
print(phash)
print(dhash)
print(whash)
print(chash)
print(type(chash))


ba3ac4c4d093d9c7
1935393977194d61
bd80818181ffb7b9
1935393977194d61,9938f078787aeae6,b83467a939597c35
<class 'imagehash.ImageMultiHash'>


In [5]:
with Image.open('pokemon-tcg-test-images/basesetcharizard.png') as img:
    resized_img = img.resize((240,330))

    test_phash = imagehash.phash(resized_img)
    test_dhash = imagehash.dhash(resized_img)
    test_chash = imagehash.crop_resistant_hash(resized_img)
    # resized_img.save(os.path.join('pokemon-tcg-test-images', 'resizedimg.png'))

    print(f"phash of real charizard: {phash - test_phash}")
    print(f"dhash of real charizard: {dhash - test_dhash}")
    for i, j in chash, test_chash:
        print(f"crop resistant hash of real charizard: {i - j}")


phash of real charizard: 22
dhash of real charizard: 16


TypeError: cannot unpack non-iterable ImageMultiHash object

In [4]:
with Image.open('pokemon-tcg-images/base1-1.png') as img2:
    alakazam_phash = imagehash.phash(img2)
    print(alakazam_phash)

    test_phash = imagehash.phash(img2)
    test_dhash = imagehash.dhash(img2)
    test_chash = imagehash.crop_resistant_hash(img2)

    print(f"phash of real charizard: {phash - test_phash}")
    print(f"dhash of real charizard: {dhash - test_dhash}")
    print(f"crop resistant hash of real charizard: {chash - test_chash}")

aa6ec0f1c393998d
phash of real charizard: 16
dhash of real charizard: 14
crop resistant hash of real charizard: 2.21875
